In [1]:
import gymnasium as gym
import mo_gymnasium as mo_gym
import numpy as np

# It follows the original Gymnasium API ...
env = mo_gym.make('minecart-v0')

/home/jijingtian/anaconda3/envs/morl/lib/python3.7/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [2]:
env = mo_gym.LinearReward(env, weight=np.array([0.8, 0.2, 0.2]))

In [2]:
def make_styled_env(env_id,seed):
    def thunk():
        env = mo_gym.make('minecart-v0')
        env = mo_gym.LinearReward(env, weight=np.array([0.1, 0.2, 0.2]))
        # env = mo_gym.MORecordEpisodeStatistics(env)
        # env = gym.wrappers.RecordEpisodeStatistics(env)
        if hasattr(env, 'seed'):
            env.seed(seed)
        env.action_space.seed(seed)
        env.observation_space.seed(seed)
        return env
    return thunk

In [3]:
env_list = [make_styled_env('minecart-v0',seed) for seed in range(3)]
# envs = mo_gym.MOSyncVectorEnv(env_list)
envs = gym.vector.SyncVectorEnv(env_list)
# envs = mo_gym.MORecordEpisodeStatistics(envs)
envs = gym.wrappers.RecordEpisodeStatistics(envs)

In [17]:
obs,_ = envs.reset()
for _ in range(100):
    next_obs, reward, done,truncted, info = envs.step(envs.action_space.sample())
    if "episode" in info.keys():
        print(info['episode'])
        for i,terminate in enumerate(info['_episode']):
            if terminate:
                print(info['episode']['r'][i])


{'r': array([-0.49599996,  0.        ,  0.        ], dtype=float32), 'l': array([29,  0,  0], dtype=int32), 't': array([0., 0., 0.], dtype=float32)}
-0.49599996
{'r': array([ 0.       , -0.8760003,  0.       ], dtype=float32), 'l': array([ 0, 69,  0], dtype=int32), 't': array([0., 0., 0.], dtype=float32)}
-0.8760003
